# Introduction
This notebook is part of a machine learning for healthcare exercise, focusing on using the Responsible AI (RAI) package to enhance clinical decision-making. The toolkit will be used to analyze diabetes progression data, with two key objectives:

1. Plan real-world action through counterfactual and causal analysis: By leveraging counterfactual example analysis and causal inferencing, we will explore decision-making strategies based on diabetes progression data to understand potential treatment paths and their impacts.

2. Assess disease progression predictions: A regression model trained on diabetes progression data will be assessed, examining its performance in predicting disease progression to inform clinical decisions.

**The goal is for you to provide non-trivial insights for clinical decision making, leveraging machine learning, paired with responsible AI tools, to improve patients outcomes in the healthcare contexts.**

Based on notebooks from the [Responsible AI toolkit](https://github.com/microsoft/responsible-ai-toolbox/tree/main/notebooks)

In [4]:
import shap
import sklearn
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


Load the diabetes dataset and specify the different types of features. Then, clean it and put it into a DataFrame with named columns.

In [5]:
data = sklearn.datasets.load_diabetes()
target_feature = 'y'
continuous_features = data.feature_names
data_df = pd.DataFrame(data.data, columns=data.feature_names)

In [8]:
data_df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


### Train a Model
After loading and cleaning the data, split the datapoints into training and test sets. Assemble separate datasets for the full sample and the test data.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data_df, data.target, test_size=0.2, random_state=7)

train_data = X_train.copy()
test_data = X_test.copy()
train_data[target_feature] = y_train
test_data[target_feature] = y_test
data.feature_names

['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

Train a RF (Random-Forest) classifier on the training data

In [10]:
model = RandomForestRegressor(random_state=0)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=0)

### Create Model and Data Insights

In [11]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights

Defining an "identity_feature". An identity feature in Microsoft's Responsible AI Toolkit helps detect and mitigate bias in sensitive attributes like race, gender, and age for fairness. 

Marking a feature as an identity feature can trigger specific analyses within the toolkit, such as:

- Fairness assessment: Evaluating if certain identity groups are disproportionately affected by model decisions
- Bias detection: Highlighting patterns of bias in model performance across identity groups
- Mitigating unfairness: Providing tools to adjust or retrain models to improve fairness with respect to the identity features

In [12]:
from responsibleai.feature_metadata import FeatureMetadata
# Add 's1' as an identity feature
feature_metadata = FeatureMetadata(
    identity_feature_name='s1', 
    categorical_features=[], 
    dropped_features=[])

RAIInsights accepts the model, the train dataset, the test dataset, the target feature string and the task type string as its arguments.

In [13]:
rai_insights = RAIInsights(
    model, 
    train_data, 
    test_data, 
    target_feature, 
    'regression',
    feature_metadata=feature_metadata)

Add the components of the toolbox that are focused on **1. Model assessment and 2. decision-making.**

### 1. Model assessment

In [ ]:
# Interpretability
rai_insights.explainer.add()
# Error Analysis
rai_insights.error_analysis.add()
# Counterfactuals: accepts total number of counterfactuals to generate, the range that their label should fall under, 
# and a list of strings of categorical feature names
rai_insights.counterfactual.add(total_CFs=20, desired_range=[50, 120])